## Convert units from BPP results 
The code below is a Python implementaton of R code from the "Mousies" script by Mario dos Reis for converting BPP units for theta and tau into Ne and divergence times in geological time units.   [https://github.com/mariodosreis/mousies/blob/master/R/analysis.R)](https://github.com/mariodosreis/mousies/blob/master/R/analysis.R).

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import toyplot
import glob

In [2]:
# get all mcmc file handles
allfiles = glob.glob("analysis-bpp/bpp00_theta_2_2000_tau_2_2000_r*.mcmc.txt")

# get headers
headers = open(allfiles[0]).readline()

# concatenate all files with header into a single file
outname = "analysis-bpp/bpp00_theta_2_2000_tau_2_2000_concat_mcmc.txt"
with open(outname, 'w') as out:
    out.write(headers)
    for mcmcfile in allfiles:
        out.write("\n".join(open(mcmcfile).readlines()[1:]))

In [3]:
# load data as a dataframe
df = pd.read_table(outname)

# print n samples
df.shape

(500000, 27)

In [4]:
# show head
df.head()

,Gen,theta_11A,theta_21B,theta_31C,theta_42A,theta_52B,theta_62C,theta_73A,theta_83B,theta_93C,...,theta_173B3C,tau_101A1B1C2A2B2C3A3B3C,tau_111A1B1C,tau_121B1C,tau_132A2B2C3A3B3C,tau_142A2B2C,tau_152B2C,tau_163A3B3C,tau_173B3C,lnL
0,10,0.001989,0.000738,0.001069,0.002951,0.002308,0.005580,0.000913,0.005583,0.003326,...,0.001435,0.000907,0.000569,0.000373,0.000835,0.000833,0.000803,0.000636,0.000379,-11412.977
1,20,0.001879,0.000957,0.001187,0.002813,0.002137,0.005929,0.001139,0.005419,0.003061,...,0.001689,0.001007,0.000584,0.000373,0.000926,0.000924,0.000897,0.000758,0.000368,-11397.858
2,30,0.001472,0.000691,0.001157,0.002352,0.002130,0.005665,0.001222,0.004466,0.002960,...,0.001656,0.000999,0.000597,0.000352,0.000904,0.000902,0.000867,0.000784,0.000357,-11376.825
3,40,0.001977,0.000613,0.001028,0.002151,0.001685,0.005218,0.001071,0.002398,0.003097,...,0.001602,0.000882,0.000516,0.000329,0.000803,0.000801,0.000785,0.000693,0.000291,-11416.800
4,50,0.002236,0.000694,0.001163,0.002439,0.002001,0.005691,0.001211,0.002892,0.003406,...,0.001850,0.000998,0.000596,0.000364,0.000909,0.000906,0.000884,0.000777,0.000352,-11362.362


In [5]:
# show all thetas
theta_names = [i for i in df.columns if "theta" in i]
df.loc[:, theta_names].columns.tolist()

['theta_11A',
 'theta_21B',
 'theta_31C',
 'theta_42A',
 'theta_52B',
 'theta_62C',
 'theta_73A',
 'theta_83B',
 'theta_93C',
 'theta_101A1B1C2A2B2C3A3B3C',
 'theta_111A1B1C',
 'theta_121B1C',
 'theta_132A2B2C3A3B3C',
 'theta_142A2B2C',
 'theta_152B2C',
 'theta_163A3B3C',
 'theta_173B3C']

In [6]:
# show all taus
tau_names = [i for i in df.columns if "tau" in i]
df.loc[:, tau_names].columns.tolist()

['tau_101A1B1C2A2B2C3A3B3C',
 'tau_111A1B1C',
 'tau_121B1C',
 'tau_132A2B2C3A3B3C',
 'tau_142A2B2C',
 'tau_152B2C',
 'tau_163A3B3C',
 'tau_173B3C']

# Priors

### Generation time
Generation time is represented by a gamma distribution with mean halfway between our estimates of 8 and 16 years. 

In [7]:
# my designated min and max range for the generation time of Malagasy Canarium
emax = 12.
emin = 24

In [8]:
# mean of prior is midway between max and min
mean = (emax + emin) / 2.

# var of prior is (range / 4)**2
var = ((emax - emin) ** 2) / 16

In [9]:
# shape and scale parameters of the gamma dist.
a = mean ** 2 / var   
b = mean / var         

In [10]:
# get 100 values evenly spaced across this gamma dist
x = np.linspace(
    ss.gamma.ppf(0.0001, a, **{'scale': 1 / b}),
    ss.gamma.ppf(0.9999, a, **{'scale': 1 / b}), 
    100)

# plot values across range of gamma
toyplot.fill(
    x,
    ss.gamma.pdf(x, a, **{'scale': 1 / b}),
    height=300, 
    width=400, 
    xlabel = "Generation time (years)",
    ylabel = "Prob. density Gamma(g|a,b)"
);

<polygon points="50.0,250.0 53.030303030303024,250.0 56.060606060606077,250.0 59.090909090909093,250.0 62.121212121212118,250.0 65.151515151515142,250.0 68.181818181818187,250.0 71.212121212121218,250.0 74.242424242424235,250.0 77.272727272727266,250.0 80.303030303030312,250.0 83.333333333333343,250.0 86.36363636363636,250.0 89.393939393939405,250.0 92.424242424242436,250.0 95.454545454545453,250.0 98.484848484848484,250.0 101.5151515151515,250.0 104.54545454545456,250.0 107.57575757575758,250.0 110.60606060606062,250.0 113.63636363636364,250.0 116.66666666666669,250.0 119.69696969696969,250.0 122.72727272727272,250.0 125.75757575757575,250.0 128.78787878787881,250.0 131.81818181818181,250.0 134.84848484848487,250.0 137.87878787878788,250.0 140.90909090909091,250.0 143.93939393939394,250.0 146.96969696969697,250.0 149.99999999999997,250.0 153.030303030303,250.0 156.06060606060606,250.0 159.09090909090912,250.0 162.12121212121212,250.0 165.15151515151516,250.0 168.18181818181819,250.0 171.21212121212125,250.0 174.24242424242425,250.0 177.27272727272728,250.0 180.30303030303031,250.0 183.33333333333334,250.0 186.36363636363637,250.0 189.39393939393938,250.0 192.42424242424241,250.0 195.45454545454544,250.0 198.48484848484847,250.0 201.51515151515147,250.0 204.54545454545459,250.0 207.57575757575759,250.0 210.60606060606062,250.0 213.63636363636363,250.0 216.66666666666669,250.0 219.69696969696972,250.0 222.72727272727275,250.0 225.75757575757578,250.0 228.78787878787881,250.0 231.81818181818181,250.0 234.84848484848484,250.0 237.87878787878788,250.0 240.90909090909091,250.0 243.93939393939391,250.0 246.96969696969694,250.0 250.00000000000003,250.0 253.03030303030303,250.0 256.06060606060606,250.0 259.09090909090907,250.0 262.12121212121218,250.0 265.15151515151513,250.0 268.18181818181824,250.0 271.21212121212119,250.0 274.24242424242425,250.0 277.27272727272725,250.0 280.30303030303031,250.0 283.33333333333331,250.0 286.36363636363637,250.0 289.39393939393938,250.0 292.42424242424244,250.0 295.45454545454544,250.0 298.4848484848485,250.0 301.51515151515156,250.0 304.54545454545456,250.0 307.57575757575756,250.0 310.60606060606062,250.0 313.63636363636368,250.0 316.66666666666669,250.0 319.69696969696969,250.0 322.72727272727275,250.0 325.75757575757575,250.0 328.78787878787875,250.0 331.81818181818181,250.0 334.84848484848482,250.0 337.87878787878793,250.0 340.90909090909088,250.0 343.93939393939394,250.0 346.969696969697,250.0 350.0,250.0 350.0,249.87751647751156 346.969696969697,249.8505096793977 343.93939393939394,249.81788640069743 340.90909090909088,249.77856120411599 337.87878787878793,249.73125837492606 334.84848484848482,249.67448301462713 331.81818181818181,249.60648873248633 328.78787878787875,249.5252417551898 325.75757575757575,249.42838132059703 322.72727272727275,249.31317628452706 319.69696969696969,249.17647795246967 316.66666666666669,249.01466925406072 313.63636363636368,248.8236105100145 310.60606060606062,248.5985822017611 307.57575757575756,248.33422534576684 304.54545454545456,248.02448029938563 301.51515151515156,247.66252508429977 298.4848484848485,247.24071460732168 295.45454545454544,246.75052248534718 292.42424242424244,246.18248753865325 289.39393939393938,245.52616739953038 286.36363636363637,244.77010208393841 283.33333333333331,243.90179078216894 280.30303030303031,242.90768552684978 277.27272727272725,241.77320577603592 274.24242424242425,240.48277828493485 271.21212121212119,239.01990690758083 268.18181818181824,237.36727714149939 265.15151515151513,235.50690027281053 262.12121212121218,233.42030186238844 259.09090909090907,231.08875899997096 256.06060606060606,228.49359020654208 253.03030303030303,225.61650105103305 250.00000000000003,222.43998743418931 246.96969696969694,218.94779705462915 243.93939393939391,215.12544779303104 240.90909090909091,210.96079962487244 237.87878787878788,206.44467421038055 234.84848484848484,201.57151354079559 231.81818181818181,196.34006599285743 228.7878787878788

In [11]:
# sample random variables from this distribution
gentime_rvs = ss.gamma.rvs(a, **{'scale': 1/b, 'random_state': 123, 'size': df.shape[0]})

### Mutation rate
Per generation mutation rate is represented by a gamma distribution with mean halfway between our estimates of 0.5 and 1.5 x 10 ^ -8. 

In [12]:
# my designated min and max range for the per gen mutation rate of Canarium (x10-8)
emax = 0.5
emin = 1.5

# mean of prior is midway between max and min
mean = (emax + emin) / 2.

# var of prior is (range / 4)**2
var = ((emax - emin) ** 2) / 16

# shape and scale parameters of the gamma dist.
a = mean ** 2 / var   
b = mean / var         

# get 100 values evenly spaced across this gamma dist
x = np.linspace(
    ss.gamma.ppf(0.0001, a, **{'scale': 1/b}),
    ss.gamma.ppf(0.9999, a, **{'scale': 1/b}), 
    100)

# plot values across range of gamma
toyplot.fill(
    x,
    ss.gamma.pdf(x, a, **{'scale': 1/b}),
    height=300, 
    width=400, 
    xlabel = "Per generation mutation rate (x 10^-8)",
    ylabel = "Prob. density Gamma(g|a,b)"
);

<polygon points="93.926437963613964,250.0 96.513039600345138,250.0 99.099641237076312,250.0 101.68624287380749,250.0 104.27284451053865,250.0 106.85944614726982,250.0 109.446047784001,250.0 112.03264942073217,250.0 114.61925105746334,250.0 117.2058526941945,250.0 119.79245433092569,250.0 122.37905596765688,250.0 124.96565760438803,250.0 127.5522592411192,250.0 130.13886087785039,250.0 132.72546251458155,250.0 135.31206415131271,250.0 137.8986657880439,250.0 140.48526742477506,250.0 143.07186906150622,250.0 145.65847069823741,250.0 148.24507233496857,250.0 150.83167397169973,250.0 153.41827560843092,250.0 156.00487724516208,250.0 158.59147888189327,250.0 161.17808051862445,250.0 163.76468215535562,250.0 166.3512837920868,250.0 168.93788542881796,250.0 171.52448706554912,250.0 174.11108870228028,250.0 176.69769033901147,250.0 179.28429197574266,250.0 181.87089361247382,250.0 184.45749524920495,250.0 187.0440968859362,250.0 189.63069852266736,250.0 192.21730015939852,250.0 194.80390179612965,250.0 197.39050343286084,250.0 199.977105069592,250.0 202.56370670632322,250.0 205.15030834305435,250.0 207.73690997978554,250.0 210.32351161651675,250.0 212.91011325324789,250.0 215.49671488997902,250.0 218.08331652671021,250.0 220.66991816344137,250.0 223.25651980017255,250.0 225.84312143690371,250.0 228.42972307363493,250.0 231.01632471036612,250.0 233.6029263470972,250.0 236.18952798382838,250.0 238.7761296205596,250.0 241.36273125729079,250.0 243.94933289402195,250.0 246.53593453075314,250.0 249.1225361674843,250.0 251.70913780421549,250.0 254.29573944094659,250.0 256.88234107767778,250.0 259.46894271440897,250.0 262.0555443511401,250.0 264.64214598787129,250.0 267.22874762460253,250.0 269.81534926133367,250.0 272.4019508980648,250.0 274.98855253479593,250.0 277.57515417152717,250.0 280.16175580825831,250.0 282.74835744498949,250.0 285.33495908172068,250.0 287.92156071845187,250.0 290.50816235518306,250.0 293.09476399191419,250.0 295.68136562864532,250.0 298.26796726537657,250.0 300.85456890210764,250.0 303.44117053883889,250.0 306.02777217557008,250.0 308.61437381230127,250.0 311.2009754490324,250.0 313.78757708576364,250.0 316.37417872249478,250.0 318.96078035922591,250.0 321.5473819959571,250.0 324.13398363268828,250.0 326.72058526941947,250.0 329.3071869061506,250.0 331.89378854288179,250.0 334.48039017961304,250.0 337.06699181634411,250.0 339.65359345307525,250.0 342.24019508980649,250.0 344.82679672653768,250.0 347.41339836326881,250.0 350.0,250.0 350.0,249.88416676071836 347.41339836326881,249.86210485804972 344.82679672653768,249.83602713796782 342.24019508980649,249.8052427876865 339.65359345307525,249.76895064109522 337.06699181634411,249.72622330541697 334.48039017961304,249.67598936233361 331.89378854288179,249.6170134831591 329.3071869061506,249.54787430020053 326.72058526941947,249.46693988359212 324.13398363268828,249.37234068572303 321.5473819959571,249.26193983515302 318.96078035922591,249.13330069001873 316.37417872249478,248.98365159892322 313.78757708576364,248.8098478668478 311.2009754490324,248.60833098652679 308.61437381230127,248.37508527385768 306.02777217557008,248.10559214121702 303.44117053883889,247.79478235692119 300.85456890210764,247.43698677434583 298.26796726537657,247.02588617203969 295.68136562864532,246.5544610278875 293.09476399191419,246.01494225689822 290.50816235518306,245.39876417381802 287.92156071845187,244.6965211979909 285.33495908172068,243.89793009718034 282.74835744498949,242.9917998666358 280.16175580825831,241.96601165519289 277.57515417152717,240.80751147549319 274.98855253479593,239.5023187622235 272.4019508980648,238.0355541599568 269.81534926133367,236.39149021739667 267.22874762460253,234.55362892140184 264.64214598787129,232.50481020282177 262.0555443511401,230.22735566460136 259.46894271440897,227.70325179538236 256.88234107767778,224.91437681076368 254.29573944094659,221.84277497890341 251.70913780421549,218.47098180499387 249.1225361674843,214.78240273737637 246.53593453075314,2

In [13]:
# sample random variables from this distribution
mutrate_rvs = ss.gamma.rvs(a, **{'scale': 1/b, 'random_state': 123, 'size': df.shape[0]})

## Transform mcmc data

In [14]:
for column in df:
    if "theta" in column:
        _, name = column.split("_")
        df["Ne_{}".format(name)] = df.loc[:, column] / ((mutrate_rvs * 1e-8) * 4)
        
    if "tau" in column:
        _, name = column.split("_")
        df["div_{}".format(name)] = (df.loc[:, column]  * gentime_rvs) / (mutrate_rvs * 1e-8)

In [15]:
# plot example transformed values for one lineage
toyplot.bars(
    np.histogram(
        df["Ne_11A"],
        bins=30,
        normed=True,    
    ),
    height=300,
    width=400,
    xmax=200000,
    ylabel="frequency",
    xlabel="Ne (effective population size)",
    label="Lineage 1A"
);

# plot example transformed values for one lineage
toyplot.bars(
    np.histogram(
        df["div_121B1C"] / 1e6,
        bins=20,
        normed=True,    
    ),
    height=300,
    width=400,
    ylabel="frequency",
    xlabel="divergence time in Mya, Lineage (1B,1C)",
    label="Lineage (1B,1C)"
);

0 50000 100000 150000 200000 Ne (effective population size) 0.00000 0.00001 0.00002 frequency Lineage 1A

0 2 4 6 divergence time in Mya, Lineage (1B,1C) 0.0 0.5 1.0 frequency Lineage (1B,1C)

In [16]:
# make a nice dataframe for divergence time summary 
divergence_times = (
 df[[col for col in df.columns if "div" in col]]
 .apply(lambda x: x / 1e6)
 .rename(columns=lambda x: x[6:])
 .describe()
 .T 
)

# save as CSV and show
divergence_times.to_csv("divtimes.csv")
divergence_times.style.format("{:.3f}")

,count,mean,std,min,25%,50%,75%,max
1A1B1C2A2B2C3A3B3C,500000.000,1.797,0.646,0.365,1.341,1.686,2.130,10.098
1A1B1C,500000.000,1.189,0.504,0.142,0.832,1.103,1.450,6.480
1B1C,500000.000,0.839,0.417,0.029,0.542,0.771,1.058,6.142
2A2B2C3A3B3C,500000.000,1.717,0.612,0.359,1.285,1.613,2.033,9.725
2A2B2C,500000.000,1.592,0.581,0.317,1.180,1.497,1.894,8.250
2B2C,500000.000,1.440,0.552,0.179,1.051,1.353,1.729,7.671
3A3B3C,500000.000,1.188,0.475,0.172,0.851,1.105,1.433,6.648
3B3C,500000.000,0.784,0.354,0.038,0.536,0.726,0.969,4.123


In [17]:
# make a nice dataframe for Ne summary 
nes = (
 df[[col for col in df.columns if "Ne" in col]]
 .rename(columns=lambda x: x.split("Ne_")[1])
 .describe()
 .T
)

# save as CSV and show as ints
nes.to_csv("Ne.csv")
nes.style.format("{:.0f}")

,count,mean,std,min,25%,50%,75%,max
11A,500000,47851,20541,6519,33324,43961,57962,390065
21B,500000,24456,13482,997,14936,21603,30719,316413
31C,500000,34596,16758,3869,22989,31116,42246,268909
42A,500000,61369,24123,11159,44095,57117,73855,338214
52B,500000,29358,12745,5264,20380,26732,35438,186183
62C,500000,167167,60579,28492,124296,156978,198394,964643
73A,500000,27145,14384,2619,17033,23701,33598,228811
83B,500000,102886,40349,12817,74452,96084,123512,720338
93C,500000,65524,29903,5649,44089,60904,81415,370538
101A1B1C2A2B2C3A3B3C,500000,56577,22358,8143,40679,52866,68303,393473
